In [1]:
import requests
from decimal import Decimal
import threading , time
def obtain(address,file_name):
    url = 'http://api.map.baidu.com/geocoder?output=json&key=f247cdb592eb43ebac6ccd27f796e2d2&location=' + str(address)
    response = requests.get(url)
    answer = response.json()
    print('得到反解数据', answer)
    lng = answer['result']['location']['lng']
    lat = answer['result']['location']['lat']
    formatted_address = answer['result']['formatted_address']
    business = answer['result']['business']
    city = answer['result']['addressComponent']['city']
    direction = answer['result']['addressComponent']['city']
    distance = answer['result']['addressComponent']['direction']
    district = answer['result']['addressComponent']['district']
    province = answer['result']['addressComponent']['province']
    street = answer['result']['addressComponent']['street']
    street_number = answer['result']['addressComponent']['street_number']
    cityCode = answer['result']['cityCode']
    s = str('%6f' % lng) + '|' + str('%6f' % lat) + '|' + str(formatted_address) + '|' + str(business) + '|' + str(
        city) + '|' + str(direction) + '|' + str(distance) + '|' + str(district) + '|' + str(province) + '|' + str(
        street) + '|' + str(street_number) + '|' + str(cityCode)
    print(s)
    if cityCode==0:
        print('外国')
        pass
    else:
        #print(s)
        fname=str(file_name)+'list'
        with open(fname, 'a+', encoding=('utf-8')) as f:
            f.write(s+'\n')

def longitude(slon,precision,elon,slat,file_name):
    while slon >= elon:
        address = '%s,%s' % (slon,slat)  # 请求用的，经度，纬度，
        obtain(address,file_name)#传入经度，纬度，获取地址地名
        slon -= precision


class Thre(threading.Thread):#继承线程中的类
    #Thre(single_list,precision,ctime)
    #temp = Thre(single_list, precision, ctime, file_name)  # 实例化类
    def __init__(self,single_list,precision,ctime,file_name):
        super(Thre,self).__init__()#重新写父类，解决多继承问题
        self.precision=precision#精度
        self.single_list=single_list#单列表
        self.ctime=ctime#当前时间
        self.file_name=file_name#文件名
    def run(self):
        print('执行线程开始时间:', self.ctime, '执行的列表:',self.single_list)
        slons, elons, slats, elats = self.single_list
        # ['42.000000', '30.000000', '105.000000', '79.800000'],  # 鸡身
        slon = Decimal(slons)  # 经度longitude开始
        elon = Decimal(elons)  # 经度结束
        slat = Decimal(slats)  # 纬度latitude开始
        elat = Decimal(elats)  # 纬度结束
        while slat >= elat:
            slat -= self.precision
            longitude(slon,self.precision, elon, slat,self.file_name)
        else:
            end_time=time.time()-self.ctime
            print('执行线程所用时间:',end_time, '执行的列表:',self.single_list)

def main():
    LogLatList=[
        ['42.000000', '30.000000', '105.000000', '79.800000'],  # 鸡身
        ['42.000000', '21.000000', '129.000000', '105.000000'],  # 鸡胸
        ['50.000000', '42.000000', '135.000000','129.000000'],  #鸡头2
        ['54.000000', '42.000000', '129.000000', '115.000000'],  #鸡头1
        ['45.500000', '42.000000', '115.000000', '105.000000'],  #鸡头补充
        ['49.200000', '42.000000', '91.500000', '79.800000'],  # 鸡屁股上
        ['42.000000', '30.000000', '79.800000', '73.400000'],  # 鸡屁股下
        ['45.500000', '42.000000', '105.000000', '91.500000'],  #鸡屁股上补充
        ['30.000000', '21.000000', '105.000000', '97.300000'],  # 鸡下
        ['42.000000', '30.000000', '97.300000', '79.800000'],  #鸡下补充
        ['21.000000', '3.000000', '129.000000', '105.000000'],  # 鸡脚 （南海）
       ]
    # ——测试使用——开始————
    precision = Decimal('1.0001000')  # 精度 0.0001000==1000米
    # ——测试使用——结束————

    #——实际获取时替换——精度自己控制————建议是 0.0001000 ————开始————
    #precision = Decimal('0.0001000')  # 精度 0.0001000==1000米
    # ——实际获取时替换——精度自己控制————建议是 0.0001000 ————结束————

    file_name=0

    thre_list=[] #线程列表
    for single_list in LogLatList:
        ctime=time.time()#当前时间
        temp=Thre(single_list,precision,ctime,file_name)#实例化类
        file_name+=1
        thre_list.append(temp)#线程列表
    for thre in thre_list:#线程列表
        thre.start()#执行单个线程

if __name__ == '__main__':
    main()


执行线程开始时间:执行线程开始时间: 1554341660.1791587 执行的列表: ['42.000000', '30.000000', '105.000000', '79.800000']
执行线程开始时间:执行线程开始时间: 1554341660.1791587 执行的列表: ['50.000000', '42.000000', '135.000000', '129.000000']
 1554341660.1791587 执行的列表: ['54.000000', '42.000000', '129.000000', '115.000000']
 1554341660.1791587 执行的列表: ['42.000000', '21.000000', '129.000000', '105.000000']
执行线程开始时间:执行线程开始时间: 1554341660.1791587 执行的列表: ['45.500000', '42.000000', '115.000000', '105.000000']
 1554341660.1791587 执行的列表: ['49.200000', '42.000000', '91.500000', '79.800000']
执行线程开始时间: 1554341660.1791587 执行的列表: ['42.000000', '30.000000', '79.800000', '73.400000']
执行线程开始时间: 1554341660.1791587 执行的列表: ['45.500000', '42.000000', '105.000000', '91.500000']
执行线程开始时间: 1554341660.1791587 执行的列表: ['30.000000', '21.000000', '105.000000', '97.300000']
执行线程开始时间:执行线程开始时间: 1554341660.1791587 执行的列表: ['42.000000', '30.000000', '97.300000', '79.800000'] 1554341660.1791587 执行的列表: ['21.000000', '3.000000', '129.000000', '105.000000']

得到反解数据 {'

得到反解数据得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 103.9999, 'lat': 39.9998}, 'formatted_address': '内蒙古自治区阿拉善盟阿拉善右旗', 'business': '', 'addressComponent': {'city': '阿拉善盟', 'direction': '', 'distance': '', 'district': '阿拉善右旗', 'province': '内蒙古自治区', 'street': '', 'street_number': ''}, 'cityCode': 230}}
103.999900|39.999800|内蒙古自治区阿拉善盟阿拉善右旗||阿拉善盟|阿拉善盟||阿拉善右旗|内蒙古自治区|||230
 {'status': 'OK', 'result': {'location': {'lng': 113.9999, 'lat': 43.4998}, 'formatted_address': '内蒙古自治区锡林郭勒盟苏尼特左旗', 'business': '', 'addressComponent': {'city': '锡林郭勒盟', 'direction': '', 'distance': '', 'district': '苏尼特左旗', 'province': '内蒙古自治区', 'street': '', 'street_number': ''}, 'cityCode': 63}}
113.999900|43.499800|内蒙古自治区锡林郭勒盟苏尼特左旗||锡林郭勒盟|锡林郭勒盟||苏尼特左旗|内蒙古自治区|||63
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 96.2999, 'lat': 39.9998}, 'formatted_address': '甘肃省酒泉市瓜州县', 'business': '', 'addressComponent': {'city': '酒泉市', 'direction': '', 'distance': '', 'district': '瓜州县', 'province': '甘肃省', 'street': '',

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 127.9999, 'lat': 48.9995}, 'formatted_address': '黑龙江省黑河市逊克县', 'business': '', 'addressComponent': {'city': '黑河市', 'direction': '', 'distance': '', 'district': '逊克县', 'province': '黑龙江省', 'street': '', 'street_number': ''}, 'cityCode': 39}}
127.999900|48.999500|黑龙江省黑河市逊克县||黑河市|黑河市||逊克县|黑龙江省|||39
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 90.4999, 'lat': 44.1995}, 'formatted_address': '新疆维吾尔自治区昌吉回族自治州木垒哈萨克自治县', 'business': '', 'addressComponent': {'city': '昌吉回族自治州', 'direction': '', 'distance': '', 'district': '木垒哈萨克自治县', 'province': '新疆维吾尔自治区', 'street': '', 'street_number': ''}, 'cityCode': 93}}
90.499900|44.199500|新疆维吾尔自治区昌吉回族自治州木垒哈萨克自治县||昌吉回族自治州|昌吉回族自治州||木垒哈萨克自治县|新疆维吾尔自治区|||93
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 133.9999, 'lat': 44.9995}, 'formatted_address': '', 'business': '', 'addressComponent': {'city': '', 'direction': '', 'distance': '', 'district': '', 'province': '', 'street': '', 'street_num

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 96.2999, 'lat': 34.9993}, 'formatted_address': '青海省玉树藏族自治州曲麻莱县', 'business': '', 'addressComponent': {'city': '玉树藏族自治州', 'direction': '', 'distance': '', 'district': '曲麻莱县', 'province': '青海省', 'street': '', 'street_number': ''}, 'cityCode': 71}}
96.299900|34.999300|青海省玉树藏族自治州曲麻莱县||玉树藏族自治州|玉树藏族自治州||曲麻莱县|青海省|||71
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 127.9999, 'lat': 12.9992}, 'formatted_address': '', 'business': '', 'addressComponent': {'city': '', 'direction': '', 'distance': '', 'district': '', 'province': '', 'street': '', 'street_number': ''}, 'cityCode': 0}}
127.999900|12.999200||||||||||0
外国
得到反解数据得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 127.9999, 'lat': 33.9992}, 'formatted_address': '', 'business': '', 'addressComponent': {'city': '', 'direction': '', 'distance': '', 'district': '', 'province': '', 'street': '', 'street_number': ''}, 'cityCode': 0}}
127.999900|33.999200||||||||||0
外国
 {'status'

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 127.9999, 'lat': 9.9989}, 'formatted_address': '', 'business': '', 'addressComponent': {'city': '', 'direction': '', 'distance': '', 'district': '', 'province': '', 'street': '', 'street_number': ''}, 'cityCode': 0}}
127.999900|9.998900||||||||||0
外国
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 102.9998, 'lat': 28.9999}, 'formatted_address': '四川省乐山市峨边彝族自治县', 'business': '', 'addressComponent': {'city': '乐山市', 'direction': '', 'distance': '', 'district': '峨边彝族自治县', 'province': '四川省', 'street': '', 'street_number': ''}, 'cityCode': 79}}
102.999800|28.999900|四川省乐山市峨边彝族自治县||乐山市|乐山市||峨边彝族自治县|四川省|||79
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 111.9997, 'lat': 43.4998}, 'formatted_address': '内蒙古自治区锡林郭勒盟二连浩特市', 'business': '', 'addressComponent': {'city': '锡林郭勒盟', 'direction': '', 'distance': '', 'district': '二连浩特市', 'province': '内蒙古自治区', 'street': '', 'street_number': ''}, 'cityCode': 63}}
111.999700|43.499800|内蒙古自治

...

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 104.9976, 'lat': 16.9996}, 'formatted_address': '', 'business': '', 'addressComponent': {'city': '', 'direction': '', 'distance': '', 'district': '', 'province': '', 'street': '', 'street_number': ''}, 'cityCode': 0}}
104.997600|16.999600||||||||||0
外国
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 104.9976, 'lat': 15.9995}, 'formatted_address': '', 'business': '', 'addressComponent': {'city': '', 'direction': '', 'distance': '', 'district': '', 'province': '', 'street': '', 'street_number': ''}, 'cityCode': 0}}
104.997600|15.999500||||||||||0
外国
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 108.998, 'lat': 28.9987}, 'formatted_address': '重庆市酉阳土家族苗族自治县', 'business': '', 'addressComponent': {'city': '重庆市', 'direction': '', 'distance': '', 'district': '酉阳土家族苗族自治县', 'province': '重庆市', 'street': '', 'street_number': ''}, 'cityCode': 132}}
108.998000|28.998700|重庆市酉阳土家族苗族自治县||重庆市|重庆市||酉阳土家族苗族自治县|重庆市|||132
得到反解数据 {'status

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 107.9979, 'lat': 35.9994}, 'formatted_address': '甘肃省庆阳市庆城县', 'business': '', 'addressComponent': {'city': '庆阳市', 'direction': '', 'distance': '', 'district': '庆城县', 'province': '甘肃省', 'street': '', 'street_number': ''}, 'cityCode': 135}}
107.997900|35.999400|甘肃省庆阳市庆城县||庆阳市|庆阳市||庆城县|甘肃省|||135
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 107.9979, 'lat': 34.9993}, 'formatted_address': '陕西省咸阳市彬县', 'business': '', 'addressComponent': {'city': '咸阳市', 'direction': '', 'distance': '', 'district': '彬县', 'province': '陕西省', 'street': '', 'street_number': ''}, 'cityCode': 323}}
107.997900|34.999300|陕西省咸阳市彬县||咸阳市|咸阳市||彬县|陕西省|||323
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 107.9979, 'lat': 33.9992}, 'formatted_address': '陕西省西安市周至县', 'business': '', 'addressComponent': {'city': '西安市', 'direction': '', 'distance': '', 'district': '周至县', 'province': '陕西省', 'street': '', 'street_number': ''}, 'cityCode': 233}}
107.997900|33.9

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 106.9978, 'lat': 31.999}, 'formatted_address': '四川省巴中市巴州区', 'business': '', 'addressComponent': {'city': '巴中市', 'direction': '', 'distance': '', 'district': '巴州区', 'province': '四川省', 'street': '', 'street_number': ''}, 'cityCode': 239}}
106.997800|31.999000|四川省巴中市巴州区||巴中市|巴中市||巴州区|四川省|||239
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 106.9978, 'lat': 30.9989}, 'formatted_address': '四川省达州市渠县', 'business': '', 'addressComponent': {'city': '达州市', 'direction': '', 'distance': '', 'district': '渠县', 'province': '四川省', 'street': '', 'street_number': ''}, 'cityCode': 369}}
106.997800|30.998900|四川省达州市渠县||达州市|达州市||渠县|四川省|||369
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 106.9978, 'lat': 29.9988}, 'formatted_address': '重庆市长寿区', 'business': '', 'addressComponent': {'city': '重庆市', 'direction': '', 'distance': '', 'district': '长寿区', 'province': '重庆市', 'street': '', 'street_number': ''}, 'cityCode': 132}}
106.997800|29.99880

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 105.9977, 'lat': 28.9987}, 'formatted_address': '重庆市江津区s106', 'business': '', 'addressComponent': {'city': '重庆市', 'direction': '', 'distance': '', 'district': '江津区', 'province': '重庆市', 'street': 's106', 'street_number': ''}, 'cityCode': 132}}
105.997700|28.998700|重庆市江津区s106||重庆市|重庆市||江津区|重庆市|s106||132
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 105.9977, 'lat': 27.9986}, 'formatted_address': '四川省泸州市古蔺县', 'business': '', 'addressComponent': {'city': '泸州市', 'direction': '', 'distance': '', 'district': '古蔺县', 'province': '四川省', 'street': '', 'street_number': ''}, 'cityCode': 331}}
105.997700|27.998600|四川省泸州市古蔺县||泸州市|泸州市||古蔺县|四川省|||331
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 105.9977, 'lat': 26.9985}, 'formatted_address': '贵州省毕节市黔西县', 'business': '', 'addressComponent': {'city': '毕节市', 'direction': '', 'distance': '', 'district': '黔西县', 'province': '贵州省', 'street': '', 'street_number': ''}, 'cityCode': 0}}
105

得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 104.9976, 'lat': 24.9983}, 'formatted_address': '贵州省黔西南布依族苗族自治州兴义市', 'business': '', 'addressComponent': {'city': '黔西南布依族苗族自治州', 'direction': '', 'distance': '', 'district': '兴义市', 'province': '贵州省', 'street': '', 'street_number': ''}, 'cityCode': 343}}
104.997600|24.998300|贵州省黔西南布依族苗族自治州兴义市||黔西南布依族苗族自治州|黔西南布依族苗族自治州||兴义市|贵州省|||343
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 104.9976, 'lat': 23.9982}, 'formatted_address': '云南省文山壮族苗族自治州广南县xh29', 'business': '', 'addressComponent': {'city': '文山壮族苗族自治州', 'direction': '', 'distance': '', 'district': '广南县', 'province': '云南省', 'street': 'xh29', 'street_number': ''}, 'cityCode': 177}}
104.997600|23.998200|云南省文山壮族苗族自治州广南县xh29||文山壮族苗族自治州|文山壮族苗族自治州||广南县|云南省|xh29||177
得到反解数据 {'status': 'OK', 'result': {'location': {'lng': 104.9976, 'lat': 22.9981}, 'formatted_address': '', 'business': '', 'addressComponent': {'city': '', 'direction': '', 'distance': '', 'district': '', 'province': '

In [2]:
import json  
from urllib.request import urlopen, quote   

import xlrd 
import requests
from datetime import datetime
def readexcel(path):
    workbook = xlrd.open_workbook(path)
    sheets = workbook.sheet_names()
    #worksheet = workbook.sheet_by_name(sheets[0])
   # worksheet = workbook.sheets()[0]
    worksheet = workbook.sheet_by_index(0)
    data=[]
    for i in range(0, worksheet.ncols):
        row = worksheet.row(i)
        data.append(worksheet.col_values(i))
    return data
data=readexcel('C:\\Users\\Administrator\\Desktop\\airstation.xlsx')
url = 'http://api.map.baidu.com/geocoder/v2/'  
a=data[0][1:]
#a=['江苏天长环保科技有限公司']
output = 'json'  
ak = 'tmdOWRA5iN76ULgPrczPYHXCS4oj4162'  
loc=[]
for i in a:  
     add = quote(i)  
     uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak #百度地理编码API  
     req = urlopen(uri)  
     res = req.read().decode()  
     temp = json.loads(res)  
     if temp['status']==0:
         temp['result']['location']['address']=i
         loc.append(temp['result']['location'])
     else:
        loc.append({'address':'江苏须百度','lng': 119.36848893835847, 'lat': 33.01379716995394})


In [5]:
import requests
from decimal import Decimal
import threading , time
import json
file_name = 'C:\\Users\\Administrator\\Desktop\\飞机站.csv'
def obtain(ad,address,file_name):
    url = 'http://api.map.baidu.com/geocoder?output=json&key=f247cdb592eb43ebac6ccd27f796e2d2&location=' + str(address)
    response = requests.get(url)
    answer = response.json()
    #print('得到反解数据', answer)
    lng = answer['result']['location']['lng']
    lat = answer['result']['location']['lat']
    formatted_address = answer['result']['formatted_address']
    business = answer['result']['business']
    city = answer['result']['addressComponent']['city']
    direction = answer['result']['addressComponent']['city']
    distance = answer['result']['addressComponent']['direction']
    district = answer['result']['addressComponent']['district']
    province = answer['result']['addressComponent']['province']
    street = answer['result']['addressComponent']['street']
    street_number = answer['result']['addressComponent']['street_number']
    cityCode = answer['result']['cityCode']
    s = str(city)
    print(s)
    if cityCode==0:
        print('外国')
        s='外国'
    fname=str(file_name)
    with open(fname, 'a+', encoding=('utf-8')) as f:
        f.write(ad+','+s+'\n')
for i in range(len(A['lat'])):
    ad=A['address'][i]
    address=str(A['lat'][i])+','+str(A['lng'][i])
    obtain(ad,address,file_name)

唐山市
哈尔滨市
长春市
南宁市
沈阳市
北京市
抚顺市
威海市
南京市
济南市
重庆市
重庆市
牡丹江市
肇庆市
大连市
上海市
广州市
齐齐哈尔市
攀枝花市
香港特别行政区
临沂市
石家庄市
北京市
南京市
抚顺市
上海市
贵港市
北京市
昆明市
青岛市
南京市
克拉玛依市
青岛市
北京市
深圳市
南充市
深圳市
伊春市
绵阳市
锦州市
成都市
郑州市
南宁市
武汉市
南京市
深圳市
贵阳市
鄂尔多斯市
青岛市
天津市
厦门市
嘉峪关市
辽源市
珠海市
东营市
海口市
广州市
福州市
安顺市
乌鲁木齐市
太原市
西双版纳傣族自治州
毕节市
外国
阜阳市
重庆市
上饶市
重庆市
揭阳市
北京市
辽源市
上海市
上海市
张家界市
乌鲁木齐市
合肥市
黄山市
汉中市
兰州市
十堰市
海南省直辖县级行政单位
外国
泸州市
济宁市
烟台市
丹东市
南京市
惠州市
湛江市
贵阳市
成都市
南昌市
长沙市
邯郸市
海南省直辖县级行政单位
外国
澳门特别行政区
北京市
盐城市
柳州市
南通市
常德市
丽江市
三亚市
上海市
兰州市
遵义市
宁波市
温州市
南京市
桂林市
上海市
银川市
运城市
北海市
厦门市
长春市
南京市
泉州市
宁波市
唐山市
香港特别行政区
北京市
南京市
包头市
景德镇市
昆明市
安顺市
福州市
赤峰市
北京市
景德镇市
南京市
乌鲁木齐市
抚顺市
太原市
阜阳市
珠海市
南京市
南京市
佛山市
福州市
福州市
南京市
天津市
呼伦贝尔市
广州市
太原市
宜昌市
南京市
南通市
南京市
吉安市
北京市
重庆市
武汉市
三亚市
连云港市
南京市
塔城地区
南京市
迪庆藏族自治州
保山市
东营市
承德市
赤峰市
泉州市
丽江市
呼和浩特市
凉山彝族自治州
大连市
秦皇岛市
拉萨市
温州市
延边朝鲜族自治州
南京市
北京市
蚌埠市
南京市
十堰市
佛山市
宁德市
福州市
吉安市
吕梁市
西宁市
玉树藏族自治州
乌鲁木齐市
保山市
黔西南布依族苗族自治州
阿克苏地区
信阳市
和田地区
宜春市
伊犁哈萨克自治州
大连市
德宏傣族景颇族自治州
文山壮族苗族自治州
恩施土家族苗族自治州
景德镇市
深圳市
淮安市
重庆市
宜昌市
洛阳市
威海市
忻州市
榆林市
赤峰市
铜仁市
外国
大庆市
重庆市
南京市
洛阳市
襄阳市
外国
香港特别行政区
长沙市
北京市
河池市


In [4]:
import pandas as pd
A=pd.DataFrame(loc)
A.to_csv('C:\\Users\\Administrator\\Desktop\\飞机坐标.csv',index=False,sep=',')